# 무신사 상품 리뷰 크롤링(crawling)

### 환경설정

In [1]:
#필요하면 설치하기
#!pip install webdriver-manager
#from selenium import webdriver
#from webdriver_manager.chrome import ChromeDriverManager
#driver = webdriver.Chrome(ChromeDriverManager().install())
#!pip install selenium

In [2]:
import warnings
warnings.filterwarnings("ignore")
from selenium import webdriver
from bs4 import BeautifulSoup
import time
import re
import time
import pandas as pd

from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from html_table_parser import parser_functions as parser

 

In [3]:
#driver = webdriver.Chrome('chromedriver.exe')
#위치: C:\Users\rlawndms\boaz_삭제절대금지
#executable_path='C:/Users/rlawndms/boaz_삭제절대금지/chromedriver.exe'
#3페이지
options = webdriver.ChromeOptions()
options.add_experimental_option("excludeSwitches", ["enable-logging"])
driver = webdriver.Chrome(options=options)
driver.get('https://www.musinsa.com/categories/item/001004?d_cat_cd=001004&brand=&list_kind=small&sort=emt_high&sub_sort=&page=3&display_cnt=90&group_sale=&exclusive_yn=&sale_goods=&timesale_yn=&ex_soldout=&kids=&color=&price1=&price2=&shoeSizeOption=&tags=&campaign_id=&includeKeywords=&measure=')
time.sleep(3)

In [4]:
 #user, gender,height,weight,item, size,content, evaluation(size_eval,bright_eval,color_eval,thick_eval)
user_list = []
gender_list =[]
height_list = []
weight_list =[]
item_list = []
size_list = []
star_list= []
content_list = []
size_eval_list =[]
bright_eval_list =[]
color_eval_list =[]
thick_eval_list =[]
#함수정의
def get_content(driver):
    #함수안에 html, soup 넣어놔야 페이지 넘어가서 바르게 긁어옴, 밖에 빼놓으면 첫페이지만 여러번 긁어진다.
    html = driver.page_source
    soup = BeautifulSoup(html,'lxml')
    for i in range(10):
    
    #profile(gender,height,weight)
    #<p class="review-profile__body_information">남성, 177cm, 85kg</p>
        profile_before = soup.find_all('p','review-profile__body_information')
        profile_after = profile_before[i].text.split(',')
        try:
            gender = profile_after[0]
            height = profile_after[1]
            weight = profile_after[2]
        except:
            gender = ''
            height = ''
            weight = ''
#user :<p class="review-profile__name">LV 2 뉴비_95f88e16</p>           
#item: #/<a href="https://www.musinsa.com/app/goods/1231416/0" class="review-goods-information__name">테이퍼드 히든 밴딩 크롭 슬랙스 [더스티 베이지]</a>
#size :<span class="review-goods-information__option">
#content

        try:
            user = soup.find_all('p','review-profile__name')[i].text
            item = soup.find_all('a','review-goods-information__name')[i].text
            # '/n' 없애고 추출하기
            size = soup.find_all('span', 'review-goods-information__option')[i].text.strip().replace('\n','')
            content = soup.find_all('div','review-contents__text')[i].text
        except:
            user = ''
            item = ''
            size = ''
            content = ''
            
        #star
#->별 5개일때:<span class="review-list__rating__active" style="width: 100%"></span>
#->별 4개일때:<span class="review-list__rating__active" style="width: 80%"></span>
        stars = driver.find_elements_by_xpath('//*[@id="reviewListFragment"]/div['+str(i+1)+']/div[3]/span/span/span')
        try:
            for j in stars:
                a =j.get_attribute('style')
                if a[7:9]=='20':
                    star = 1
                elif a[7:9]=='40':
                    star = 2
                elif a[7:9]=='60':
                    star = 3
                elif a[7:9]=='80':
                    star = 4
                else:
                    star = 5
        except:
            star = ''
      

    #evaluation
        evaluation = soup.find_all('div', 'review-evaluation')
        try:
            size_eval = evaluation[i].find_all('span')[0].text
            bright_eval = evaluation[i].find_all('span')[1].text
            color_eval = evaluation[i].find_all('span')[2].text
            thick_eval = evaluation[i].find_all('span')[3].text
        except:
            size_eval = ''
            bright_eval = ''
            color_eval = ''
            thick_eval = ''

#user,gender,height,weight,item, size,content,star,size_eval,bright_eval,color_eval,thick_eval
        
        user_list.append(user)
        gender_list.append(gender)
        height_list.append(height)
        weight_list.append(weight)
        item_list.append(item)
        size_list.append(size)
        content_list.append(content)
        star_list.append(star)
        size_eval_list.append(size_eval)
        bright_eval_list.append(bright_eval)
        color_eval_list.append(color_eval)
        thick_eval_list.append(thick_eval)
  
   
        


In [5]:
#버튼 누르기 함수정의
def move_next(driver):    
    for i in range(4):
        get_content(driver)
        #페이지 2,3,4,5 넘어가기
        driver.find_element_by_css_selector('#reviewListFragment > div.nslist_bottom > div.pagination.textRight > div > a:nth-child(' + 
                                            str(int(4) + int(i)) + ')').send_keys(Keys.ENTER)
        time.sleep(2)
    get_content(driver)
    
#그다음 화살표'>'버튼누르기: (6,7,8...)있는 페이지로 넘어가기   
def move_arrow(driver):
    driver.find_element_by_css_selector('#reviewListFragment > div.nslist_bottom > div.pagination.textRight > div > a.fa.fa-angle-right.paging-btn.btn.next').send_keys(Keys.ENTER)
    

### 크롤링시작

In [8]:
#후기순 90개 자동으로 클릭하기
#뒤로가기 참고: https://m.blog.naver.com/mathesis_time/221970036063
# #searchList > li:nth-child(1) > div.li_inner > div.list_img > a > img
# #searchList > li:nth-child(2) > div.li_inner > div.list_img > a > img
# #searchList > li:nth-child(90) > div.li_inner > div.list_img > a > img
 #90개: 90
a = 12
#상품개수에 따라 숫자 바꾸기
for i in range(87,91,1):
    driver.find_element_by_css_selector('#searchList > li:nth-child(' +
                                            str(int(i)) +
                                            ') > div.li_inner > div.list_img > a > img').click()
    time.sleep(2)
        
        #사이즈표
    try:
        html = driver.page_source
        soup = BeautifulSoup(html,'html.parser')
        figure = soup.find('table',{'class':'table_th_grey'})
        p = parser.make2d(figure)
        figure_df = pd.DataFrame(data = p[1:],columns = p[0])
        figure_df.drop([0,1],inplace = True)
        print(figure_df)
    except:
        print("사이즈표 오류발생")
        
        #리뷰개수
    try:
        reviewNum = driver.find_element_by_xpath('//*[@id="estimate_style"]')
        reviewNum = reviewNum.text
        reviewNum = re.sub(r'[^0-9]','',reviewNum)
        reviewNum = int(reviewNum)
        print(reviewNum)
    except:
        print("리뷰개수 오류발생")
        

    user_list = []
    gender_list =[]
    height_list = []
    weight_list =[]
    item_list = []
    size_list = []
    star_list= []
    content_list = []
    size_eval_list =[]
    bright_eval_list =[]
    color_eval_list =[]
    thick_eval_list =[]
        
        #스타일후기리뷰 100개넘으면 진행,아니라면 진행 X
    if reviewNum >= 100:
        #크롤링
        #10:500개, 2:100개
        for i in range(2):
            try:
                move_next(driver)
                move_arrow(driver)
        #move_next(driver)
            except:
                time.sleep(2)
                    

                    
                    #뒤로가기
        driver.back()
        time.sleep(2)
    else:
        driver.back()
            
    time.sleep(2)    
    globals()["df"+str(a)] = pd.DataFrame({'user':user_list,
                                           'gender':gender_list,
                                           'height':height_list,
                                           'weight':weight_list,
                                            'item':item_list,
                                            'size':size_list,
                                            'star':star_list,
                                            'content':content_list,
                                            'size_eval':size_eval_list,
                                            'bright_eval':bright_eval_list,
                                            'color_eval':color_eval_list,
                                            'thick_eval':thick_eval_list})          
    #사이즈표와 리뷰 merge
    time.sleep(2) 
    globals()["merge_df"+str(a)] = pd.merge(globals()["df"+str(a)],figure_df,how = 'left',left_on = 'size',right_on = 'cm')
    a = a + 1            


   cm  총장 어깨너비 가슴단면 소매길이
2   S  63   55   56   60
3   M  67   58   59   62
4   L  69   60   61   63
5  XL  71   62   63   64
235
    cm  총장 어깨너비 가슴단면 소매길이
2    M  68   64   64   58
3    L  70   66   66   59
4   XL  72   68   68   60
5  2XL  74   70   70   61
562
  cm  총장 어깨너비 가슴단면 소매길이
2  M  71   58   67   62
3  L  73   60   69   64
72
  cm  총장 어깨너비 가슴단면 소매길이
2  S  68   62   70   61
3  M  70   64   72   62
4  L  72   66   74   63
81


In [11]:
#print(merge_df19.shape)#3,4,10없음 
merge_df13.head(1)
#merge

,user,gender,height,weight,item,size,star,content,size_eval,bright_eval,color_eval,thick_eval,cm,총장,어깨너비,가슴단면,소매길이
0,LV 4 파란색깃발,남성,168cm,63kg,섹션 후드 블랙 [쭈리/기모 선택] YMHD6556 /YMHD6610,(쭈리)블랙/M,5,스트릿한 무드로 연출할 때 좋아용두께감도 적당해서 내년 봄까지 잘 입을래요,보통이에요,보통이에요,보통이에요,보통이에요,NaN,NaN,NaN,NaN,NaN


In [12]:
df85_90=pd.concat([merge_df10,merge_df11,merge_df12,merge_df13,merge_df14,
                 merge_df15],
                axis =0)
print(df85_90.shape)
df85_90.drop_duplicates(subset = None,keep = 'first',
                        inplace = True,ignore_index = True)
print(df85_90.shape)


(379, 17)
(351, 17)


In [13]:
df = pd.read_csv('후드후기순3페이지_221206.csv')
#df.head()
df.shape

(4018, 17)

In [14]:
#1~84
dftotal = pd.concat([df85_90,df], axis = 0)
print(dftotal.shape)
dftotal.drop_duplicates(subset = None,keep = 'first',
                        inplace = True,ignore_index = True)
print(dftotal.shape)
dftotal.to_csv('후드후기순3페이지_221206.csv',index = False)

(4369, 17)
(4028, 17)


In [15]:
df = pd.read_csv('후드후기순3페이지_221206.csv')
#df.head()
df.shape

(4028, 17)

# 비고

In [5]:
# 할인쿠폰 창 없애기->그냥 수동으로 했음

* 참고
* 글로벌, 로컬: https://www.nemonein.xyz/2021/02/4908/